In [ ]:
#ESM
!pip install fair-esm
import esm
import torch
import pandas as pd

def make_esm_representations(df,col, model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    #df = pd.read_excel(input_file_name)
    peptides_list = df[col].tolist()

    esm_model, alphabet = model
    esm_model = esm_model.to(device)
    batch_converter = alphabet.get_batch_converter()

    peptides_list = [("", peptides_list[i]) for i in range(len(peptides_list))]
    batch_labels, batch_strs, batch_tokens = batch_converter(peptides_list)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

    # Extract per-residue representations (on CPU or CUDA)
    results = esm_model(batch_tokens.to(device), repr_layers=[6], return_contacts=True)
    token_representations = results["representations"][6]

    # Generate per-sequence representations via averaging
    sequence_representations = []
    for i, tokens_len in enumerate(batch_lens):
        sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))

    sequence_representations = torch.stack(sequence_representations)
    df = pd.DataFrame(sequence_representations.cpu().detach().numpy())
    return df
    #df.to_excel(output_file_name, index=False)
    #print(f'{output_file_name} file made')

def pretrained_model(dim):
    assert dim in [5120,2560,1280,640,480,320]
    if dim==5120:
        return esm.pretrained.esm2_t48_15B_UR50D()
    elif dim==2560:
        return esm.pretrained.esm2_t36_3B_UR50D()
    elif dim==1280:
        return esm.pretrained.esm2_t33_650M_UR50D()
    elif dim==640:
        return esm.pretrained.esm2_t30_150M_UR50D()
    elif dim==480:
        return esm.pretrained.esm2_t12_35M_UR50D()
    elif dim==320:
        return esm.pretrained.esm2_t6_8M_UR50D()




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 3.5 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv("/content/Mp.csv")
print(df.head())

                                            Sequence
0  MSIITDVYAREVLDSRGNPTLEVEVYTESGAFGRGMVPSGASTGEH...
1  MAITKIHARSVYDSRGNPTVEVDVVTETGLHRAIVPSGASTGQHEA...
2  MAREFSLEKTRNIGIMAHVDAGKTTTTERILYYTGKIHKIGETHEG...
3  MSIITDVYAREVLDSRGNPTLEVEVYTESGAFGRGMVPSGASTGEH...
4  MAKVIGIDLGTTNSCLAISENGQTKVIENAEGARTTPSVIAYLDGG...


In [ ]:
chunk_size = 1
results2 = []
c = 0
for chunk in pd.read_csv("/content/Mp.csv", chunksize=chunk_size):
    # Apply make_esm_representations function to the chunk
    chunk_results = make_esm_representations(chunk,'Sequence', model=pretrained_model(320))
    results2.append(chunk_results)
    print(c)
    c+=1
    # Clear memory (optional, depending on your RAM capacity)
    del chunk, chunk_results





0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266


In [ ]:
result_df = pd.concat(results2)

import numpy as np

result_array = result_df.to_numpy()
np.save('MP_esm2V2.npy', result_array)

print(result_df.head())

        0         1         2         3         4         5         6    \
0  0.083978  0.072194  0.087015  0.146480  0.179757  0.053801  0.040912   
0 -0.027097  0.001796  0.092432  0.147086  0.209930  0.130138 -0.000399   
0 -0.031078  0.021842  0.042063  0.197593  0.243343  0.038649  0.079137   
0  0.054546  0.064043  0.085998  0.170648  0.168747  0.069392  0.043424   
0 -0.148942  0.031161  0.098035  0.157785  0.199050  0.167786  0.168320   

        7         8         9    ...       310       311       312       313  \
0 -0.168640  0.075406 -0.190192  ... -0.020650  0.115753 -0.105518 -0.085044   
0 -0.096349 -0.005315 -0.077787  ... -0.054319  0.109403 -0.082497 -0.018667   
0 -0.092343 -0.057382 -0.059526  ...  0.042319  0.065393 -0.130362  0.137950   
0 -0.181351  0.070163 -0.186088  ... -0.040261  0.115168 -0.104292 -0.082216   
0 -0.053220 -0.139958 -0.062488  ...  0.136002 -0.020356 -0.067747  0.237618   

        314       315       316       317       318       319  
0  0